# Modelling a perfectly spherical star in python


# Libraries

In [1]:
import math
import re
import numpy as np

from debugpy.launcher.winapi import kernel32

# Useful constants
Makes life easier.

In [3]:
e: float = math.e
G: float = 6.6743*10**(-11)
pi: float = math.pi
c: float = 299792458
h: float = 6.6260*10**(-34)
sigma: float = 5.6703*10**(-8)
k: float = 1.38 * 10**(-23)
R: float = 8.3145
epsilon_0: float = 8.854*10**(-12)
e_charge: float = 1.602*10**(-19)
r_0: float = 1.4*10**(-15)

# Defining elements

In [2]:
class element:
    #Proton number
    Z: int
    #Nucleon number
    N: int

# Maxwell-Boltzmann cumulative distribution
This will be used to determine what proportion of particles near the surface have enough KE to overcome GPE near the surface. I will then cut down those numbers by first cutting it in half to model the particles heading back into the star. I will later attempt to make a formula linking density, gas temperature and particle speed to further find what percentage of particles will collide with the surrounding gas particles and not escape.

In order to do this, we need to find the integral of the Maxwell-Boltzmann distribution function, so that we can find the cumulative distribution function.

$$f(v)=(\frac{m}{2 \pi k_B T})^{3/2}4 \pi v^2 e^{-\frac{mv^2}{2 k_B T}}$$

$$\int f(v)dv=\int (\frac{m}{2 \pi k_B T})^{3/2}4 \pi v^2 e^{-\frac{mv^2}{2 k_B T}} dv$$

$$P(v_0, v_1, m)=4 \pi (\frac{m}{2 \pi k T})^{\frac{3}{2}}(-\frac{kT}{m}ve^{-\frac{mv^2}{2kT}}+\frac{\sqrt{2 \pi k^3 T^3}}{2m}\erf({\sqrt{\frac{m}{2kT}}v}))\Bigg|_{v_0}^{v_1} $$
This function is given as a function of velocity. Converting it to be a function of $KE$ would be too messy.

To make this easier to read, I'll break this down into a few functions. Along with this,any particle with energy higher than the minimum energy will be accepted, so $v_1=\infty$, and so the new euqation can be written:

$$P(v, m)=1-k_0(m)(k_1(v)k_2(m, v)+k_3(m)\erf({\sqrt{\frac{m}{2kT}}v})) $$
Where:
$$k_0(m) = 4 \pi (\frac{m}{2 \pi k T})^{\frac{3}{2}}$$
$$k_1(v) = -\frac{kT}{m}v$$
$$k_2(m, v) = e^{-\frac{mv^2}{2kT}}$$
$$k_3(m) = \frac{\sqrt{2 \pi k^3 T^3}}{2m}$$

This function does not have a concise analytical solution. We will have to take approximations to be able to do this.
We will use the math library to estimate the error function (erf).



In [4]:
def BoltzMannProbabliltyDistribution(E_min: float, mass: float, T: float) -> float:
    v_min = math.sqrt((2*E_min)/(mass))
    k0 = 4 * pi * math.sqrt(((mass)/(2 * pi * k * T)**2))
    k1 = (-k * T * v_min)/mass
    k2 = math.exp((-mass*v_min*v_min)/(2*k*T))
    k3 = (math.sqrt(2*pi*k*k*k*T*T*T))/(2*mass)
    erf = math.erf(math.sqrt((mass)/(2*k*T))*v_min)

    probability = 1-k0*(k1*k2+k3+erf)

    return probability

# Coulomb Force, Nuclear Radius and Fusion
Like charges repel by the equation:
$$F = \frac{Q_1Q_2}{4\pi\epsilon_0r^2}$$
And the potential energy of a charged particle at distance r is:
$$U = \frac{Q_1Q_2}{4\pi\epsilon_0r}$$
So with these equations, we know the charge required for a charged particle $Q_1$ to come within $r$ distance of another charged particle $Q_2$. In order to find the energy required to get two nuclei to fuse, we can assume they are evenly charged spheres of radius $r_1$ and $r_2$, and charge $Q_1$ and $Q_2$, and in order to fuse, they need to 'touch', i.e. the distance between their centers is $r_{total} = r_1+r_2$
$$E=\frac{Q_1Q_1}{4\pi\epsilon_0(r_1+r_2)}$$
This is the total KE of the two particles. However, we can take the relative velocity between them, and that velocity will still follow the Boltzmann distribution. However, the average relative velocity will on average be slightly larger than the average velocity for a singular particle. I'll try and work it out later, but I can't be bothered.

Now, to find the nuclear radius, we use the formula:
$$r=r_0A^{1/3}$$
Where $r_0$ is some constant.

In [ ]:
def minFusionEnergy(A1: int, A2: int, protonNum1: int, protonNum2: int) -> float:
    r1 = r_0 * math.cbrt(A1)
    r2 = r_0 * math.cbrt(A2)
    energy = ((protonNum1*protonNum2) * (e_charge**2))/(4*pi*epsilon_0*(r1+r2))
    return energy

# The star model

The star will be modelled as a sphere, with many layers of differing compsosition. Each layer will consist of several element, at uniform average temperature. By the nuclear fusion probability function, the amount of that layer that fuses will be calculated, and the new materials will be mixed into the layer. Heavier elements will move towards the core. Descending layers will also take on a larger thickness as the radius grows smaller.

The mass of the star will be calculated by summing the masses of all the layers, as well as adding the mass from the kinetic energy of the molecules.

The star will lose mass via radiation and particles escaping via evaporation.

We can find the power output of the star by using the Stefan-Boltzmann law, assuming the star is a perfect black body.
$$P = \sigma A T^4$$

Evaporation can also be found by finding the proportion of particles at the surface which have sufficient energy to escape the gravitiational field of the sun. This is modelled by using the Maxwell-Boltzmann distribution to find the proportion of particles at the surface of the sun with sufficient energy to escape.

In [ ]:
class starLayer:
    def __init__(self, mass: float, composition: [element], temperature: float, radius_0: float, radius_1: float):
        self.mass = mass
        self.composition = composition
        self.temperature = temperature

In [ ]:
class Star:
    def __init__(self, starLayers: [starLayer]):
        self.starLayers = starLayers


# Nuclear Fusion and Quantum Tunneling

Nuclear fusion is the process by which smaller nuclei with low binding energies fuse into larger nuclei, with lager binding energies and release energy during this process. In order to fuse, nuclei need to touch, however the Coulomb force would mean that for the fusion of hydrogen, classical mechanics would suggest that a temperature on the order of $10^{10}$~$10^{11}$°K would be required, many orders of magnitude higher than the temperature at the core of the sun.

Quantum tunnelling is the effect where a particle which does not have enough energy to overcome a potential energy barrier e.g. an electric field or a solid object, but is still able to regardless. This small increase to the probability of overcoming the electric field between nuclei is what allows smaller stars which cannot undergo thermonuclear fusion to remain stable.

The rate at which fusion occurs is given by the equation:

$$r \simeq n_a n_b (\frac{8}{\pi m_R})^{1/2}\frac{S(E_0)}{(kT)^{3/2}}\frac{\Delta \sqrt{\pi}}{2}\exp{-\frac{3E_0}{kT}}$$

Where

$$\Delta = 4\sqrt{\frac{E_0 k T}{3}}$$
$$E_0 = \sqrt[3]{\frac{(\pi Z_a Z_b e^2)^2m_R(kT)^2}{2 \hbar^2}}$$

$S(E_0)$ is the S-Factor. However, since I cannot currently find a full table of the relevant S-Factors, I will instead use already calculated rates.

<small>The above equations were derived using equations found in sources 1 and 2<small>

# PP chain, CNO cycles and Helium capture
To model the actual fusion of the particles themselves, and to calculate what products they would form would be much too complicated. Because of this, I will predefine certain reactions that are allowed to happen in my star model.

I won't, however, bother to add the hot CNO cycles (for now), since they typically only occur duing supernovae, and that is far beyond the scope of this model.

## PP-I Chain

$$^1_1H + ^1_1H \rightarrow ^2_2He + \beta^+ + v_e$$

$$^2_2He + ^1_1H \rightarrow ^3_2He + \gamma$$

$$^3_2He + ^3_2He \rightarrow ^4_2He + 2^1_1H$$

---
## PP-II Chain

$$^1_1H + ^1_1H \rightarrow ^2_2He + \beta^+ + v_e$$

$$^2_2He + ^1_1H \rightarrow ^3_2He + \gamma$$

$$^3_2He + ^4_2He \rightarrow ^7_4be + \gamma$$

$$^7_4Be + ^0_0e^- \rightarrow ^7_3Li + v_e$$

$$^7_3Li + ^1_1H \rightarrow 2^4_2He$$


---
## PP-III Chain

$$^1_1H + ^1_1H \rightarrow ^2_2He + \beta^+ + v_e$$

$$^2_2He + ^1_1H \rightarrow ^3_2He + \gamma$$

$$^3_2He + ^4_2He \rightarrow ^7_4He + \gamma$$

$$^7_4Be + ^1_1H \rightarrow ^8_5B + \gamma$$

$$^8_5B \rightarrow ^8_4Be + ^0_0e^+ + v_e$$

$$^8_4Be \rightarrow 2^4_2He$$

In the rates data, the last two reactions are listed as one, this is probably because its half life is ~0.77s.

---
## PP-IV Chain

$$^1_1H + ^1_1H \rightarrow ^2_2He + \beta^+ + v_e$$

$$^2_2He + ^1_1H \rightarrow ^3_2He + \gamma$$

$$^3_2He + ^1_1H \rightarrow ^4_2He + ^0_0e^+ + v_e$$


---

## Helium capture

$$^4_2He + ^4_2He \rightarrow ^8_4Be$$
$$^4_2He + ^8_4Be\rightarrow ^{12}_6C$$
$$^4_2He + ^{12}_6C\rightarrow ^{16}_8O$$
$$^4_2He + ^{16}_8O\rightarrow ^{20}_10Ne$$
$$^4_2He + ^{20}_10Ne\rightarrow ^{24}_12Mg$$
$$...$$

This occurs all the way up to $^{56}_{28}Fe$, and then stops due to binding energy going down again.

---

### **CNO-I Cycle (Main Branch)**


$${}^{12}\text{C} + p \rightarrow {}^{13}\text{N} + \gamma$$

$${}^{13}\text{N} \rightarrow {}^{13}\text{C} + e^+ + \nu_e$$

$${}^{13}\text{C} + p \rightarrow {}^{14}\text{N} + \gamma$$

$${}^{14}\text{N} + p \rightarrow {}^{15}\text{O} + \gamma$$

$${}^{15}\text{O} \rightarrow {}^{15}\text{N} + e^+ + \nu_e$$

$${}^{15}\text{N} + p \rightarrow {}^{12}\text{C} + {}^4\text{He}$$


---

### **CNO-II Cycle**


$${}^{12}\text{C} + p \rightarrow {}^{13}\text{N} + \gamma$$

$${}^{13}\text{N} \rightarrow {}^{13}\text{C} + e^+ + \nu_e$$

$${}^{13}\text{C} + p \rightarrow {}^{14}\text{N} + \gamma$$

$${}^{14}\text{N} + p \rightarrow {}^{15}\text{O} + \gamma$$

$${}^{15}\text{O} \rightarrow {}^{15}\text{N} + e^+ + \nu_e$$

$${}^{15}\text{N} + p \rightarrow {}^{12}\text{C} + {}^4\text{He}$$

$${}^{12}\text{C} + p \rightarrow {}^{13}\text{N} + \gamma$$


---

### **CNO-III Cycle**



$${}^{12}\text{C} + p \rightarrow {}^{13}\text{N} + \gamma$$

$${}^{13}\text{N} \rightarrow {}^{13}\text{C} + e^+ + \nu_e$$

$${}^{13}\text{C} + p \rightarrow {}^{14}\text{N} + \gamma$$

$${}^{14}\text{N} + p \rightarrow {}^{15}\text{O} + \gamma$$

$${}^{15}\text{O} \rightarrow {}^{15}\text{N} + e^+ + \nu_e$$

$${}^{15}\text{N} + p \rightarrow {}^{12}\text{C} + {}^4\text{He}$$


---

### **CNO-IV Cycle**


$${}^{12}\text{C} + p \rightarrow {}^{13}\text{N} + \gamma$$

$${}^{13}\text{N} \rightarrow {}^{13}\text{C} + e^+ + \nu_e$$

$${}^{13}\text{C} + p \rightarrow {}^{14}\text{N} + \gamma$$

$${}^{14}\text{N} + p \rightarrow {}^{15}\text{O} + \gamma$$

$${}^{15}\text{O} \rightarrow {}^{15}\text{N} + e^+ + \nu_e$$

$${}^{15}\text{N} + p \rightarrow {}^{12}\text{C} + {}^4\text{He}$$





# Reading Reaclib1 files

In [ ]:
#This code was not written by me. It was written by Deepseek

def parse_reaclib1_file(filename):
    """
    Parses a Reaclib1 file and extracts the reaction details and coefficients.
    """
    reactions = []
    with open(filename, 'r') as file:
        lines = file.readlines()

        # Skip the header (chapter number)
        i = 1  # Start reading from the second line

        while i < len(lines):
            # First line of the set entry
            line1 = lines[i].strip()
            if not line1:  # Skip empty lines
                i += 1
                continue

            # Extract reactants and products
            reactants = [line1[5:10].strip(), line1[10:15].strip(), line1[15:20].strip()]
            products = [line1[20:25].strip(), line1[25:30].strip(), line1[30:35].strip()]

            # Extract set label, rate type, reverse flag, and Q value
            set_label = line1[43:47].strip() if len(line1) >= 47 else ''
            rate_type = line1[47] if len(line1) >= 48 else ''
            reverse_flag = line1[48] if len(line1) >= 49 else ''
            q_value = float(line1[52:64]) if len(line1) >= 64 else 0.0

            # Second line of the set entry (first four coefficients)
            line2 = lines[i + 1].strip() if i + 1 < len(lines) else ''
            # Split line2 into coefficients using 'e+' or 'e-' as delimiters
            coeffs_line2 = []
            if line2:
                # Use a regular expression to split the line into valid scientific notation strings
                import re
                parts = re.findall(r'[-+]?\d*\.\d+[eE][-+]?\d+', line2)
                coeffs_line2 = [float(part) for part in parts]

            # Third line of the set entry (last three coefficients)
            line3 = lines[i + 2].strip() if i + 2 < len(lines) else ''
            # Split line3 into coefficients using 'e+' or 'e-' as delimiters
            coeffs_line3 = []
            if line3:
                # Use a regular expression to split the line into valid scientific notation strings
                import re
                parts = re.findall(r'[-+]?\d*\.\d+[eE][-+]?\d+', line3)
                coeffs_line3 = [float(part) for part in parts]

            # Combine all coefficients
            coefficients = coeffs_line2 + coeffs_line3
            if len(coefficients) < 7:
                coefficients.extend([0.0] * (7 - len(coefficients)))  # Pad with zeros if necessary

            # Store the reaction details and coefficients
            reaction = {
                'reactants': reactants,
                'products': products,
                'set_label': set_label,
                'rate_type': rate_type,
                'reverse_flag': reverse_flag,
                'q_value': q_value,
                'coefficients': coefficients
            }
            reactions.append(reaction)

            # Move to the next set entry
            i += 3

    return reactions


def calculate_reaction_rate(T9, coefficients):
    """
    Calculates the reaction rate at a given temperature T9 (in 10^9 K).
    """
    a0, a1, a2, a3, a4, a5, a6 = coefficients
    log_rate = a0 + a1 / T9 + a2 / T9 ** (1 / 3) + a3 * T9 ** (1 / 3) + a4 * T9 + a5 * T9 ** (5 / 3) + a6 * np.log(T9)
    return np.exp(log_rate)

# Sources used:

The sources are *not* given in chronological order of access.

1. https://web.archive.org/web/20170115214447/https://zuserver2.star.ucl.ac.uk/~idh/PHAS2112/Lectures/Current/Part7.pdf

2. https://en.wikipedia.org/wiki/Stellar_nucleosynthesis#cite_note-40

3. https://www.youtube.com/@DrPhysicsA (All of nuclear playlist)

4. https://t2.lanl.gov/nis/data/astro/ for S-Factor experimental data

5. https://reaclib.jinaweb.org/popularRates.php for rates data



# How AI was used in this project

ChatGPT was used in the generation of the LaTEX displaying the CNO-I through to CNO-IV cycle.

ChatGPT was used to *double check* my equation for the rate of reaction between species A and specied B. The equation in the form that I give is the original that I derived using the given equations in sources 1 and 2.

Deepseek was used to write the code to parse data from Reaclib1 files.

ChatGPT was used in *no other aspect of this project*.